In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pointbiserialr
from sampling import pps, accept_reject

PROPERTY = 'fire_size'

dfo = pd.read_csv('./data/FW_Veg_Rem_Combined.csv')

cont_columns_prefix = ['Prec_pre_', "Temp_pre_", "Wind_pre_", "Hum_pre_"]
columns_suffix = [7, 15, 30]

weather_columns = [ x + str(y) for x in cont_columns_prefix for y in columns_suffix]

def preprocess_missing_weather_data(df):
    df_subsample = df[df['Temp_pre_7'] == -1]
    df_clean = df[df['Temp_pre_7'] != -1]
    mean_weather_data = df_clean.groupby(['state', 'discovery_month'])[weather_columns].mean().reset_index()
    df_subsample.drop(weather_columns, axis=1, inplace=True)
    df_subsample = df_subsample.merge(mean_weather_data)
    df_clean = df_clean.append(df_subsample, ignore_index=True)
    return df_clean

df = preprocess_missing_weather_data(dfo)

causes = df.stat_cause_descr.unique()
bayesian_net_columns_cont = ['remoteness', 'latitude', 'longitude', 'Temp_pre_7', 'Temp_pre_15', 'Temp_pre_30', 'Prec_pre_30', 'Hum_pre_7', 'Hum_pre_15', 'Hum_pre_30']

/home/patrik-sh/Documents/ITMO_Classes/Multivariate/tasks/task3/venv/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [24]:
ress= []

for col in bayesian_net_columns_cont:
    row = { 'property': col }
    for cause in causes:
        df[cause] = df.stat_cause_descr.apply(lambda x: x == cause)
        r,p = pointbiserialr(df[cause], df[col])
        row[cause] = r
    ress.append(row)

df_res = pd.DataFrame(ress)
print(df_res)

      property  Missing/Undefined     Arson  Debris Burning  Campfire  \
0   remoteness          -0.038411 -0.090711       -0.138068 -0.004670   
1     latitude          -0.146258 -0.051947       -0.104620  0.017823   
2    longitude           0.073169  0.137974        0.196612  0.003758   
3   Temp_pre_7           0.008170 -0.095068       -0.156220 -0.034331   
4  Temp_pre_15           0.011799 -0.097339       -0.157080 -0.033186   
5  Temp_pre_30           0.018885 -0.100629       -0.152301 -0.031213   
6  Prec_pre_30          -0.013030  0.045313        0.032301  0.006559   
7    Hum_pre_7           0.036048  0.056756        0.077787 -0.001316   
8   Hum_pre_15           0.040397  0.061290        0.092317  0.001692   
9   Hum_pre_30           0.048636  0.067344        0.129945  0.005554   

   Miscellaneous  Fireworks  Lightning  Equipment Use  Children   Smoking  \
0       0.057982   0.024309   0.201943       0.082258 -0.019140 -0.018302   
1       0.008374   0.041025   0.308865    